# Google Earth Engine Download Example

This notebook demonstrates how you can view, clip and download GIS Images from the Earth Engine Dataset


In this example a Region from the YKDelta was studied and multiple download links were created to download GeoTiff files for this specific region
For future projects I suggest that the Export.image.toDrive is utilized to store these images on the cloud to prevent a large data requirement on the computer
In addition, the MODIS dataset should be used as the bands in this data set will be essential for most of the Hazard predictions

 Import Required Libraries

In [1]:
import ee
import geemap
import pandas as pd
from IPython.display import Image

Authenticate Google Earth Engine

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AZEOvhVO_TDeO_6IQ30Oe3MvdrYYocW86O7QeFPvUGSScfAlp14GjWkRECQ

Successfully saved authorization token.


# Importing Data

- Elevation, Slope, Aspect, NDVI, Temperature, Soil Moisture and Precipitation download links were produced
- For NDVI, Temperature, and Soil Moisture a yearly average TIFF was downloaded for each year from 2000-2021.

## Elevation

In [15]:
#Alaska boundary
alaska_bound = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS','AK'))

#create section of Ykdelta
lon = 62.83
lat = -153
poi = ee.Geometry.Point(lon,lat)
poi_reprojected = poi.transform('EPSG:4269')
box = [-165.5,61,-161,64]
roi =  ee.Geometry.Rectangle(box)

In [4]:
#Import Digital Elevation Model
Dataset = ee.Image('USGS/3DEP/10m')
Elevation = Dataset.select('elevation')

#change alaska boundary to proper coordinate system
alaska_bound_reprojected = alaska_bound.geometry().transform('EPSG:4269')

#clip elevation to just alaska
Elevation_Alaska = Elevation.clip(alaska_bound_reprojected)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [20]:
Elevation_Alaska_reprojected = Elevation_Alaska.reproject(crs='WGS84')

#get download link
link = Elevation_Alaska.getDownloadURL({
    'scale' : 130,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'elevation'
})
print(link)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0748ca9ba7ea5db7ca177b441d088d9a-2bd5c9ef4a9387f97900b246e9f52aa8:getPixels


## Slope

- Slope is extracted from the previous imported Digital Elevation Module

In [21]:
Slope_Alaska = ee.Terrain.slope(Elevation_Alaska)


url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [22]:
#get download link

link = Slope_Alaska.getDownloadURL({
    'scale' : 130,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'slope'
})
print(link)

url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/bad57b5aae9aae14a9a057286334d8ad-b81b6cd2d78da5e3bba08c3442dee273:getPixels


## Aspect
- Aspect is also extracted from the DEM

In [23]:
Aspect_Alaska = ee.Terrain.aspect(Elevation_Alaska)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [24]:
#get download link

link = Aspect_Alaska.getDownloadURL({
    'scale' : 130,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'aspect'
})
print(link)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ba1cd7ca67024d65d689d0d30d988651-bfe95e2752b9c92bcf905e408e362fe4:getPixels


## NDVI

In [25]:
#NDVI
Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate('2018-01-01','2018-02-01')

NDVI = Dataset.select('NDVI')
#NDVI_Alaska = NDVI.clip('alaska_bound_reprojected')
NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
url = NDVI_Alaska.getThumbUrl({
   'min': -2000, 'max': 10000 ,'region':alaska_bound.geometry(), 'dimensions':2048,
   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

In [30]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'

    Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate(Start_Date,End_Date).select('NDVI')
    NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
    #get download link
    link = NDVI_Alaska.getDownloadURL({
        'scale' : 110,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'NDVI_' + str(i)     
    })
    print(Start_Date +": " + link)

url = NDVI_Alaska.getThumbUrl({
    'min': -2000, 'max': 10000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)
    

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/7aab900a1b6d89867354556c620fe046-c3e8489b7f35f3cd37d463c86e8c708e:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/33463a9976e7dbe7c541b8ca88cb3a1a-7b2c48c2f8e523d08adadbc2176f15f8:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e23f76d32f8558fa1427b37b2812847a-b28000f0b353496de4be5e13d34e3246:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a25136a56e6ab92ec9c69ecc97df5c45-195e85c535a78c56466a9346903c59d7:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/4f9efcf61f1b1003530adc8efafb8978-d089cb6bb9a9ad7137ba9016762539ae:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e22d2c8ee9822fd9a86864b28a93c948-70c270fb158ffb5a66fc22a

## Temperature

- Normally this wouldn't be downloaded from Earth Engine as there are more sufficient data sources; however for proof of concept these values were also used
- This specific temperature dataset recorded the max temperature of each month. An average of this was then taken for each year

In [37]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('tmmx')
    avg_maxTemp = Dataset.mean().clip(alaska_bound)
    link = avg_maxTemp.getDownloadURL({
        'scale' : 180,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'avgmaxTemp_' + str(i)     
    })
    print(Start_Date +": " + link)
url = avg_maxTemp.getThumbUrl({
    'min': -670, 'max': 576 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c1f8111f8d18e00c7a237da38822382a-a945d45de11c9a53787d71aa2163784d:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/cb0c99a375e4a5e99e1dd2f2e91508f3-b9b2fe3d242e2e5787cf2156849cabfc:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/82bccc88489a9490b05115037f2fcdd0-a2e7b9dab95771bcd4e7b8eba0c9c6e4:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/5e45b75af0ef29e7d0f8811d541364c7-ae01d1e5cf34303231a7738f52ea11d4:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/37f2fd45e9bbacccfe3609a4d2f71ef9-1d1f4a89086f7f48ab32dc28722dc5dc:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/aa3dfc780dec8b218097c7da094e6bec-033dba25137517d6d564501

## Soil Moisture
- average soil moisture per year

In [39]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('soil')
    soil = Dataset.mean().clip(alaska_bound)
    link = soil.getDownloadURL({
        'scale' : 180,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'soil_' + str(i)     
    })
    print(Start_Date +": " + link)
url = soil.getThumbUrl({
    'min': -0, 'max': 9000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2f7a6d0c68548dd0819ed406c6f09378-0c65a50455d6169b923c15f15dea4c98:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/74fd1a8e6e3000e50abbcc60bb39e46e-71da6c3d2544543725ed64d5a9040ce7:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/26933bf30f37fbeecf4f2cc4cb36b664-41548c9e302d54cb222043a98cd31f27:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/30a99d807e9bc7f2eff96b8768ad3e06-1c0468d75913d5193ac5f7b73149f104:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ee729c5e5e31479a367e41e9c9909e22-7386dd99257b8614ec6786b75d540846:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fd6f5dbeda9e31243ad7bdb2bef3780a-43f8f2c643ad5f9154189e4

## Precipitation
- total precipitation per year

In [40]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('pr')
    precip = Dataset.sum().clip(alaska_bound)
    link = precip.getDownloadURL({
        'scale' : 180,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'precip_' + str(i)     
    })
    print(Start_Date +": " + link)
url = precip.getThumbUrl({
    'min': -0, 'max': 9000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3dadac574e78c4799310a63fb89bc0fe-945ad37331f5f8cab27d79b007c1eafa:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/1969d27b6cf75645b2d6206f4053d662-8154f3e475f295756d2817d0b871f458:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ea531a843ba8d90e1a4744e9e93ef8c3-6a24d08c646c92b1982d846a53d5885d:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ddfed7f9252b81ef7520e3f3cf12b0bd-be48d035a82e573145992d67afe9cdf7:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/bcf8b5578703cda3b4ed8338ddab296b-0bbae583523a1eda7ad0aa0171041f5a:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fb42447e2a20b25d46336f38771c2544-8edf866b8cd794a54666a38